# Lab 5 — Profiling (NYPD Hate Crimes)

This notebook profiles the NYPD hate crimes dataset and uses the helper functions in `hate_crimes.py`:

- `load_hate_crimes(path)`
- `clean_hate_crimes(df)`
- `summarize_by_year(df)`

The goal is to understand data quality issues and confirm key assumptions before writing tests.


In [14]:
import pandas as pd

from hate_crimes import load_hate_crimes, clean_hate_crimes, summarize_by_year

CSV_PATH = "NYPD_Hate_Crimes_20260220.csv"

pd.set_option("display.max_columns", None)


In [15]:
df_raw = load_hate_crimes(CSV_PATH)
df_raw.shape


(4029, 14)

In [16]:
df_raw.head()


,Full Complaint ID,Complaint Year Number,Month Number,Record Create Date,Complaint Precinct Code,Patrol Borough Name,County,Law Code Category Description,Offense Description,PD Code Description,Bias Motive Description,Offense Category,Arrest Date,Arrest Id
0,201906012119317,2019,1,01/23/2019,60,PATROL BORO BKLYN SOUTH,KINGS,FELONY,MISCELLANEOUS PENAL LAW,AGGRAVATED HARASSMENT 1,ANTI-JEWISH,Religion/Religious Practice,NaN,NaN
1,201906012175717,2019,2,02/25/2019,60,PATROL BORO BKLYN SOUTH,KINGS,FELONY,MISCELLANEOUS PENAL LAW,AGGRAVATED HARASSMENT 1,ANTI-JEWISH,Religion/Religious Practice,NaN,NaN
2,201906012180117,2019,2,02/27/2019,60,PATROL BORO BKLYN SOUTH,KINGS,FELONY,MISCELLANEOUS PENAL LAW,AGGRAVATED HARASSMENT 1,ANTI-JEWISH,Religion/Religious Practice,NaN,NaN
3,201906012273417,2019,4,04/16/2019,60,PATROL BORO BKLYN SOUTH,KINGS,FELONY,MISCELLANEOUS PENAL LAW,AGGRAVATED HARASSMENT 1,ANTI-JEWISH,Religion/Religious Practice,NaN,NaN
4,201906012413717,2019,6,06/20/2019,60,PATROL BORO BKLYN SOUTH,KINGS,FELONY,MISCELLANEOUS PENAL LAW,AGGRAVATED HARASSMENT 1,ANTI-JEWISH,Religion/Religious Practice,NaN,NaN


In [17]:
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4029 entries, 0 to 4028
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Full Complaint ID              4029 non-null   int64  
 1   Complaint Year Number          4029 non-null   int64  
 2   Month Number                   4029 non-null   int64  
 3   Record Create Date             4029 non-null   object 
 4   Complaint Precinct Code        4029 non-null   int64  
 5   Patrol Borough Name            4029 non-null   object 
 6   County                         4029 non-null   object 
 7   Law Code Category Description  4029 non-null   object 
 8   Offense Description            4029 non-null   object 
 9   PD Code Description            4029 non-null   object 
 10  Bias Motive Description        4029 non-null   object 
 11  Offense Category               4029 non-null   object 
 12  Arrest Date                    0 non-null      f

In [18]:
(df_raw.isna().mean().sort_values(ascending=False) * 100).round(2).head(25)



Arrest Date                      100.00
Arrest Id                         56.64
Full Complaint ID                  0.00
Complaint Year Number              0.00
Month Number                       0.00
Record Create Date                 0.00
Complaint Precinct Code            0.00
Patrol Borough Name                0.00
County                             0.00
Law Code Category Description      0.00
Offense Description                0.00
PD Code Description                0.00
Bias Motive Description            0.00
Offense Category                   0.00
dtype: float64

In [19]:
df = clean_hate_crimes(df_raw)
df.shape


(4029, 14)

In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4029 entries, 0 to 4028
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Full Complaint ID              4029 non-null   int64         
 1   Complaint Year Number          4029 non-null   int64         
 2   Month Number                   4029 non-null   int64         
 3   Record Create Date             4029 non-null   datetime64[ns]
 4   Complaint Precinct Code        4029 non-null   int64         
 5   Patrol Borough Name            4029 non-null   object        
 6   County                         4029 non-null   object        
 7   Law Code Category Description  4029 non-null   object        
 8   Offense Description            4029 non-null   object        
 9   PD Code Description            4029 non-null   object        
 10  Bias Motive Description        4029 non-null   object        
 11  Offense Category 

In [21]:
pd.api.types.is_datetime64_any_dtype(df["Record Create Date"])


True

In [22]:
df["Month Number"].min(), df["Month Number"].max()


(np.int64(1), np.int64(12))

In [23]:
df["Complaint Year Number"].min(), df["Complaint Year Number"].max()


(np.int64(2019), np.int64(2025))

In [24]:
"Bias Motive Description" in df.columns, (df["Bias Motive Description"].isna().mean() * 100).round(2)


(True, np.float64(0.0))

In [25]:
df["Offense Category"].value_counts().head(15)


Offense Category
Religion/Religious Practice           2199
Race/Color                             846
Sexual Orientation                     552
Ethnicity/National Origin/Ancestry     273
Gender                                 153
Age                                      5
Disability                               1
Name: count, dtype: int64

In [26]:
df["Full Complaint ID"].nunique(), len(df), len(df) - df["Full Complaint ID"].nunique()


(3711, 4029, 318)

In [27]:
summary = summarize_by_year(df)
summary.head(10)


Complaint Year Number
2019    447
2020    284
2021    580
2022    672
2023    758
2024    714
2025    574
Name: Full Complaint ID, dtype: int64

Open `df` in Data Wrangler and review:

- column types after cleaning
- missing values (especially arrest-related fields)
- distributions for categorical fields (Offense Category, Bias Motive Description)
- outliers for numeric fields (if any)

In [ ]:
df


# 5 Takeaways

1) The dataset is small (far below 200,000 rows), so it is safe to load into memory for profiling and for unit tests.

2) Arrest-related fields have heavy missingness. “Arrest Date” is entirely missing (or near entirely missing), and “Arrest Id” also has a high missing rate. Arrest-based analysis is limited without another data source.

3) After cleaning, “Record Create Date” can be treated as a proper datetime field. This makes time-based analysis (trends by year/month) feasible.

4) “Month Number” stays within 1–12 after cleaning, and “Complaint Year Number” stays in a reasonable recent range. Grouping by year (and month) is reliable.

5) “Full Complaint ID” is not a unique row identifier (there are duplicates). For stable aggregation, summary tables (like counts by year) are more appropriate than assuming each row has a unique ID.
